In [1]:
import tweepy
import random
import time
#from interactive_conditional_samples import interact_model
import gpt_2_simple as gpt2
from textblob import TextBlob

/home/jonathan/miniconda3/envs/mj/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/jonathan/miniconda3/envs/mj/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/jonathan/miniconda3/envs/mj/lib/python3.5/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/jonathan/miniconda3/envs/mj/lib/python3.5/s

In [2]:
def pick_topic():
    topic = random.choice(['evolution', 'gun control', 'existence of god', 'healthcare'
                          , 'climate change', 'communism'])
    return topic

def search(topic, api):
    results = api.search(q=topic, lang='en', result_type='recent', count=1, tweet_mode='extended')
#     for res in results:
#         if 
    return results
    
def generate_responses(text,sess, gpt2):
    print('Generating responses')
    responses = gpt2.generate(sess,
                              length=280,
                              temperature=0.8,
                              prefix=text+' ||| ',
                              nsamples=5,
                              batch_size=1,
                              return_as_list=True)
    print(responses)
    return responses

def select_response(responses):
    parsed = []
    for response in responses:
        try:
            response = response.split(' ||| ')[1]
        except:
            continue
        response = response[:240]
        response = response.split('<endoftext>')[0]
        before, sep, after = response.rpartition('.')
        response = before + sep
        parsed += [response]

    parsed = [x for x in parsed if 'emoticon' not in x and 'forum' not in x and x != '']
    sent_parsed = [[x,TextBlob(x).sentiment.polarity] for x in parsed]
    parsed.sort(key = lambda x: x[1], reverse = True)

    return parsed[0]

def reply(tweet, api,sess, gpt2):
    user = tweet._json['user']['screen_name']
    tweet_id = tweet._json['id']
    text = tweet._json['full_text']
    responses = generate_responses(text,sess, gpt2)
    reply_text = select_response(responses)
    
    api.update_status('@' + user + ' ' + reply_text, in_reply_to_status_id=tweet_id)
    return text, reply_text

In [3]:
sess = gpt2.start_tf_sess()
gpt2.load_gpt2(sess, run_name='run1')

Loading checkpoint checkpoint/run1/model-5047
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from checkpoint/run1/model-5047


In [4]:
def main():
    auth = tweepy.OAuthHandler('5W4GMKnmBPmz73ofTbMSrBX5O', 'v3liOsP8k73qnqasUUbJAD0VaXyHJN6zy6tr2xdjdmuIgrjj0D')
    auth.set_access_token('977955567846281216-8YsIaPvhBtk3vQSZ5thkEvbiTTm39ro', 'h7JcHEceUN5PX8NuKYFA9f665DLMUSKpNizT1ZqFod2ei')

    api = tweepy.API(auth)
#     sess = gpt2.start_tf_sess()
#     gpt2.load_gpt2(sess, run_name='run1')
    while True:
        try:
            topic = pick_topic()
            tweet = search(topic, api)[0]
            text, reply_text = reply(tweet, api,sess, gpt2)
            print(topic)
            print('tweet: ', text)
            print('##########################')
            print('reply: ', reply_text)
            time.sleep(int(random.random()*120+300))
        except KeyboardInterrupt:
            break
        except:
            continue
        

In [5]:
main()

Generating responses
["@VinceBenvenuto @HopliteMagazine @TXPatriot36 @iheartmindy @StephenKing Government shouldn't be involved in Healthcare, Government is the reason College is expensive and the Reason Healthcare cost so much. Majority of medical breakthrough is done in America. Also Pharma companies do R&amp;D and other countries reverse engineer meds and its cheap US pays |||   Sorry, but your post is wrong.    I did not make that argument, I said that the majority of medical breakthroughs in the US are done in America. I also said that in the same article, that there are many other countries that do.      I know I don't think its easy to prove that, but I can certainly say that I have been involved in it.       Do you have any link to support your claim?      First, it is a contract between the parties. It is not a competition.     It was clearly very clear. It is a contract between the parties.     I think I've already made that argument, you don't have to prove it.       If ther

communism
tweet:  RT @jeremycorbyn: If I was proposing the creation of our NHS today, the Conservatives would call it health communism.
##########################
reply:    Well, if you are proposing it, we do not have to agree on it because after all, we live in a democracy. And democracy cannot be everything, it is not even a correct system yet, just a system.
Generating responses
['RT @realjediman2: And just like that, the media had gone silent about the #SaugusHighShooting when it was discovered that the shooter is As… |||   So much for "not believing the news". Now that\'s not good. <endoftext>   Sure, it\'s your business but it\'s so easy for you to check some facts that you can\'t even get the logic of other people\'s arguments. It\'s clear that you\'ve got no clue. My point is that the guy who committed the shooting was obviously not a law enforcement officer. There\'s no evidence that he was a criminal. There\'s no evidence that he utilized a gun. No photo, no mug shot. He did

#christian
tweet:  RT @yhwhuniversity: #Christian persecution is sadly at the top of all official statistics made, but theres also others like these #Hindus b…
##########################
reply:    Silence? What's your point?    Every human being is indeed a person.    No. Of course not. That's a casual dismissal of the very real possibility that we come up with a hybrid human - a hybrid T.O.C.
Generating responses
['If this isn\'t an indictment of our healthcare system, I don\'t know what is.\nI genuinely hope this person, and all other people in desperate need of transplants, gets the medical assistance they need. https://t.co/1WdOQw4Az5 |||  NO, that\'s not an indictment of medical access to abortions. It\'s an indictment of the faulty system and general inability to bring an honest review.   http://www.dailytelegraph.co.uk/news/article3977700/Images/women-slaves-must-have-garbage-disposal-used-yet-again.html <endoftext> I just don\'t understand what you are thinking here.   Anybody 

gay marriage
tweet:  The United States of America are Liars! For they hate God with its Abominable Law of Gay Marriage! In The Mighty Name of Jesus!!
##########################
reply:      I stay out of the gay marriage biz! I don't care! I'd be a hypocrite to spend my life in the gay marriage biz!     That's a very good point. But I'm not the one to judge where others are involved. That's not my problem.
Generating responses
["RT @yaf: Abortion is violence, plain and simple. https://t.co/s2kunW9xWj |||   Hey, here's an example of a civil union, thanks for grabbing my attention!  http://www.prop8.org/dtd/images/sboy/images/Twinengine_31822.jpg  Big difference. I'm not sure what the point is here, but hey, I'll stick with it anyway.  http://www.prop8.org/dtd/images/sboy/images/Twinengine_31822.jpg <endoftext>   Look, you still have a long ways to go, but I don't see any of your posts having that much effect. I'll keep at this one, though.      I see no evidence of the moon exploding. I'

evolution
tweet:  RT @MistressDove: Also found one of those "History and Evolution" type videos.

Honestly seeing all of this is really cool.
https://t.co/GE…
##########################
reply:    Ok, so in the above example, the result of the mutation is that all animals, not just humans, will have to evolve. To me this means that the rate of evolution will increase, or that it will decrease in frequency.
Generating responses
["OMG God answered my prayer..now that I believe in the existence of God..God can you please let me win a lottery without even having to buy in..Please God! #thanks |||   You really are the devil aren't you.      He replied me by saying you don't know how he knew I had expectations. He also quoted scripture from other people.     Well it's not like I'm paying you personally. I also want to say that I support the government in what it does. I agree that we need to create a new Constitution that is as good as any that was ever written. I'm saying that the government

...

abortion
tweet:  RT @HongKongHermit: This ballbag harassed women entering abortion clinics, to the extent he got a criminal record for it. He‘s the same vul…
##########################
reply:    Well, that is a bit of an oblique reference, but anyway, hmmm, like I said, you don't seem to be able to get much right.  I've been saying the same thing for a while. Not really sure how many times to I should say it.
Generating responses
["When Dulcy is hired to “herd bats” for a famous writer with dementia she doesn’t know her job description includes danger. https://t.co/17edpMlSx3  #Christian #CozyMystery #Romance |||   You can't have a no stick without a no line anywhere     I would agree: I'm saying you cannot have a no line without a no line anywhere. If you can have a no line without a no line anywhere it is WAY too good!     And you are not saying that?      Just because we don't have a stick, doesn't mean we can't have a no line without a no line anywhere.       If so, then we have a 

gay marriage
tweet:  I'm going to be a bit more shallow here, so please excuse me...

Nothing would give me greater pride or pleasure, than if an openly gay candidate, who is in a same-sex marriage, beat the pants off Trump! 

The #MAGAts' heads would collectively explode &amp; it would be glorious! https://t.co/tglBojrbly
##########################
reply:     No, because it's not. It's not. So I don't see why you have to prove something.    So you're saying 6 of 10 people would vote for it?    Yeah, that's a great example of gay marriage being harmful to the ratings.
Generating responses
['RT @jeremycorbyn: If I was proposing the creation of our NHS today, the Conservatives would call it health communism. |||   Ah! You know, in Germany they shoot people who commit crimes at intervals instead of charging them when they have been found through the courts to have committed a crime. It works there! <endoftext>   The parameters   I really am not trying to justify abortion. It is not my arg

climate change
tweet:  RT @anthonypesec: Sweden's central bank dumped Australian bonds because we're “not known for good climate work".

Australia is becoming a p…
##########################
reply:    France's central bank has dumped about $15 billion of sovereign bonds, after its Economic and Financial Analysis Office (EFFO) concluded Australia had become a leading carbon emitter.  http://news.national.net.
Generating responses
["RT @PinkNews: Same-sex marriage in Sweden and Denmark has reduced the number of lesbians and gay men dying by suicide by almost half https:… |||       Not only have gay people been able to have same-sex marriage, they have been able to do it for a long, long while. There are no laws against it. People have been able to have same-sex marriage for decades, and many countries already have same-sex marriage. This is only the beginning.    Do you see how it's apples and oranges just to jump on this bandwagon? <endoftext>      I'm going to deal with this because Ev

climate change
tweet:  RT @QuentinDempster: Dear Mr Murdoch @rupertmurdoch (b. Melbourne 1931) the country of your birth is now in extremis with the fires of clim…
##########################
reply:   A bit of misdirection. I never said I was a citizen of one country and lived in another, I said I was a citizen of one country.
Generating responses
['RT @TheBirmingham6: Rob Delaney : "The NHS is the greatest thing I\'ve ever experienced in my life"\n\nOf course if Corbyn suggested the NHS n… |||   I never said it was the greatest thing I have ever experienced. I\'m just saying it is NOT! <endoftext>   My father left Spain about 40 years ago and never returned. My mother is also of Spanish descent, but she has never married. They have three grown children from before that are living in California right now. |||   It might be in Germany, because you\'re in the wrong country.      Well, the US is also a country. Brits are in Austria and Swedes are in Sweden. Sounds like a great country. <en

communism
tweet:  @chelahhorsdal German Nazism = Russian Communism - this is the effect of the criminal and genocidal systems of leftist groups, which also today follow the same patterns ...
on which Hitler and Stalin modeled ...
on Marx, Engels, etc ... https://t.co/sk2Fikneyz
##########################
reply:     It can be hard to find those. The only discussion about the subject that I saw was in my last post where I had to search about 30 times to find the post.


In [11]:
auth = tweepy.OAuthHandler('5W4GMKnmBPmz73ofTbMSrBX5O', 'v3liOsP8k73qnqasUUbJAD0VaXyHJN6zy6tr2xdjdmuIgrjj0D')
auth.set_access_token('977955567846281216-8YsIaPvhBtk3vQSZ5thkEvbiTTm39ro', 'h7JcHEceUN5PX8NuKYFA9f665DLMUSKpNizT1ZqFod2ei')

api = tweepy.API(auth)

In [16]:
s = search(' Im gay and i thought marriage was stupid because its for procreating. Laws could have been changed for rights.', api)

In [15]:
api.update_status('@' + 'thedancingMilo' + ' ' + 'Marriage is not a hate crime - it is only discrimination against same-sex couples, by the state of Alaska. To be fair, no one would be forced to marry one anti-gay couple, but they would be forced to marry a gay couple.', in_reply_to_status_id=1195915746720784384)

Status(source_url='https://github.com/screaming-frog/mechanical_jerk', in_reply_to_status_id=1195915746720784384, coordinates=None, id=1195918375551156224, created_at=datetime.datetime(2019, 11, 17, 4, 15, 34), in_reply_to_screen_name='thedancingMilo', in_reply_to_user_id=1148514557255651328, entities={'user_mentions': [{'screen_name': 'thedancingMilo', 'id_str': '1148514557255651328', 'id': 1148514557255651328, 'name': 'Thedancingdeplorable', 'indices': [0, 15]}], 'urls': [{'url': 'https://t.co/ZkM1XO1mZm', 'expanded_url': 'https://twitter.com/i/web/status/1195918375551156224', 'indices': [116, 139], 'display_url': 'twitter.com/i/web/status/1…'}], 'symbols': [], 'hashtags': []}, text='@thedancingMilo Marriage is not a hate crime - it is only discrimination against same-sex couples, by the state of… https://t.co/ZkM1XO1mZm', retweet_count=0, retweeted=False, in_reply_to_status_id_str='1195915746720784384', place=None, _json={'in_reply_to_status_id': 1195915746720784384, 'in_reply_to_us

In [17]:
s[0]._json['id']

1195915746720784384